In [1]:
import datasets
import pandas as pd


aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']

# for aspect in aspects:
#     df1 = pd.read_excel(f'../../chatgpt/outputs/main_data_batch_gold_results.xlsx', sheet_name=aspect)
#     df2 = pd.read_excel(f'../../chatgpt/outputs/main_data_batch_silver_results.xlsx', sheet_name=aspect)

#     chatgpt_df = pd.concat([df1, df2])

#     chatgpt_df = chatgpt_df [chatgpt_df[f'{aspect}_label'] == chatgpt_df[f'chatgpt_{aspect}_score']]

#     print(aspect, len(chatgpt_df['id'].unique()))


#     for x in chatgpt_df[f'chatgpt_{aspect}_rationale'].head(5):
#         print(x)


In [7]:


# sampled_test_set = test_set.sample(n=100, random_state=42)

In [8]:
# for aspect in aspects:
#     sampled_test_set[f"{aspect}_rationale_correctness"] = None
#     sampled_test_set[f"{aspect}_rationale_relevance"] = None


# sampled_test_set.to_csv('human_rationale_evaluation.csv', index=False)

In [19]:
import json
import datasets
import pandas as pd

# Load test set
test_set = datasets.load_dataset("boda/review_evaluation_automatic_labels", name="all", split="test").to_pandas()

# Load GPT-4 samples
gpt_4_samples = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/analysis/rationale_evaluation/human_rationale_evaluation.csv')
gpt_4_samples = gpt_4_samples.head(25)

# Normalize review_point
def normalize(text):
    return str(text).strip().lower()

gpt_4_samples['norm_review_point'] = gpt_4_samples['review_point'].apply(normalize)
test_set['norm_review_point'] = test_set['review_point'].apply(normalize)

# Merge on normalized review_point
matched_df = pd.merge(
    gpt_4_samples, test_set,
    on='norm_review_point',
    suffixes=('_gpt4', '_test'),
    how='inner'
)

# Load LLaMA rationales
llama_rationales_path = '/home/abdelrahman.sadallah/mbzuai/review_rewrite/inference/evalute_outputs/adapters/Llama-3.1-8B-Instruct/score_rationale/instruction/all/step-0/review_evaluation_automatic_labels/predictions_all_test.jsonl'
with open(llama_rationales_path, 'r') as f:
    llama_data = [json.loads(line) for line in f]

# Get indices for LLaMA rationales and corresponding test_set rows
matched_indices = matched_df['id_test'].apply(lambda id_: test_set[test_set['id'] == id_].index[0]).tolist()
filtered_llama_rationales = [llama_data[i] for i in matched_indices]

# Convert filtered LLaMA rationales to DataFrame
llama_df = pd.DataFrame(filtered_llama_rationales)

# Get the corresponding test_set rows
matched_test_rows = test_set.iloc[matched_indices].reset_index(drop=True)

# Concatenate test_set rows with the LLaMA rationales side-by-side
combined_df = pd.concat([matched_test_rows, llama_df], axis=1)

# Optional: Save to CSV
combined_df.to_csv("matched_test_llama_rationales.csv", index=False)

print("✅ Saved combined DataFrame with LLaMA rationales and test set columns.")


✅ Saved combined DataFrame with LLaMA rationales and test set columns.


In [20]:
combined_df

,paper_id,venue,focused_review,review_point,id,chatgpt_actionability_score,chatgpt_actionability_rationale,chatgpt_helpfulness_score,chatgpt_helpfulness_rationale,chatgpt_grounding_specificity_score,...,__index_level_0__,norm_review_point,actionability_label,grounding_specificity_label,verifiability_label,helpfulness_label,actionability_rationale,grounding_specificity_rationale,verifiability_rationale,helpfulness_rationale
0,NIPS_2018_202,NIPS_2018,- My major concern with the paper is that the ...,1) above then the problem reduces to subspace ...,6284,2,The review point makes an observation about th...,2,The review comment suggests a potential approa...,1,...,4501,1) above then the problem reduces to subspace ...,3,2,1,3,The review point suggests that the problem can...,The comment refers to a specific part of the p...,The review point suggests that the problem can...,The review comment suggests a potential approa...
1,ICLR_2023_2171,ICLR_2023,1. All the tasks in this paper seem based on g...,1. All the tasks in this paper seem based on g...,3504,3,The comment suggests that the tasks in the pap...,4,The review comment identifies a potential issu...,3,...,4658,1. all the tasks in this paper seem based on g...,3,3,2,3,The review point suggests that the tasks in th...,The comment suggests that the tasks in the pap...,The review point claims that all tasks in the ...,The review comment identifies a potential limi...
2,ICLR_2023_1400,ICLR_2023,- While the paper shows improvements on CIFAR ...,- The authors should add some details on how t...,3701,5,The review comment explicitly instructs the au...,4,The comment identifies a specific area where t...,3,...,2777,- the authors should add some details on how t...,5,3,1,4,The review point explicitly instructs the auth...,The comment suggests adding details on how to ...,The review point suggests that the authors sho...,The review comment identifies a specific area ...
3,ICLR_2022_733,ICLR_2022,Weakness 1 The number of experts is limited to...,2 I am wondering if this mechanism can be used...,2700,2,The review comment poses a question about whet...,2,The comment raises a question about the applic...,1,...,5967,2 i am wondering if this mechanism can be used...,1,2,X,2,The review point raises a question about the a...,The comment raises a question about the applic...,The review point is a question seeking clarifi...,The review comment raises a question about the...
4,ICLR_2021_2180,ICLR_2021,Weakness: 1. The authors should provide justif...,"2. The authors mention: ""The typical MTTP base...",2190,5,The review point provides a clear and explicit...,5,The review comment provides a constructive sug...,5,...,7016,"2. the authors mention: ""the typical mttp base...",5,5,4,4,The review point suggests that the authors sho...,The comment is fully grounded as it explicitly...,The review point suggests that the authors sho...,The review comment identifies a potential weak...
5,ICLR_2022_1250,ICLR_2022,1. The authors claimed that “we develop a unif...,3. The experiments lack the comparison to stat...,3033,3,The comment points out that the experiments do...,3,The comment highlights a specific shortcoming ...,3,...,8361,3. the experiments lack the comparison to stat...,3,4,1,3,The review point highlights a lack of comparis...,"The comment addresses the experiments section,...",The review point claims that the experiments l...,The review comment identifies a significant ga...
6,Mq5cyRMGlD,EMNLP_2023,While the paper addresses an important problem...,"3. Despite the existence of ample work, the se...",1594,3,The comment explicitly encourages the authors ...,3,The review comment identifies a specific weakn...,3,...,7916,"3. despite the existence of ample work, the se...",5,3,2,4,The review point explicitly encourages the aut...,The comment addresses the selection of baselin...,The review point claims that the selection of ...,The review comment identifies a limitation in ...
7,NIPS_2018_544,NIPS_2018,- the